d # 1. JSON parsing
1. JSON 스키마 정의
2. 스키마 기반 Spark DataFrame 생성 및 TempView Table 생성
3. Delta Table 생성
3. TempView의 데이터를 wragling하여 Delta Table에 insert 


## 1. ADLS2 폴더 및 파일 확인
- april/study_result/544/790/... 의 형태를 갖는 디렉토리 구조를 갖고 있음

In [2]:
dbutils.fs.ls("/mnt/cdpocmount/april/study_result/544/790/93414/1046/")

Out[6]: [FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1082058/', name='1082058/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1082989/', name='1082989/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1133446/', name='1133446/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1138539/', name='1138539/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1141134/', name='1141134/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1141839/', name='1141839/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1143402/', name='1143402/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1144675/', name='1144675/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1149917/', name='1149917/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1157546/', name='1157546/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1157641/', name='1157641/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1168285/', name='1168285/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1181584/', name='1181584/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1183951/', name='1183951/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1193544/', name='1193544/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1195806/', name='1195806/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1200042/', name='1200042/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1202212/', name='1202212/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1207250/', name='1207250/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1207313/', name='1207313/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1210408/', name='1210408/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1218079/', name='1218079/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1222482/', name='1222482/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1223704/', name='1223704/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1224817/', name='1224817/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1227542/', name='1227542/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1231651/', name='1231651/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1231751/', name='1231751/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1232680/', name='1232680/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1232719/', name='1232719/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1236387/', name='1236387/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1239841/', name='1239841/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1240641/', name='1240641/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1240942/', name='1240942/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/1046/1242010/', name='1242010/', size=0),
 FileInfo(path='dbfs:/mnt/cdpocmount/april/study_result/544/790/93414/10

## 2. JSON 스키마 정의

In [4]:
%python 
from pyspark.sql.functions import *
from pyspark.sql.types import *

shadowing_1046_schema = StructType([
  StructField("sem_id", LongType(), True),
  StructField("top_cors_id", LongType(), True),
  StructField("com_id", LongType(), True),
  StructField("g_seq", LongType(), True),
  StructField("app_ver", StringType(), True),
  StructField("serial", StringType(), True),
  StructField("mac_address", StringType(), True),
  StructField("manufacturer", StringType(), True),
  StructField("model", StringType(), True),
  StructField("submit_seq", StringType(), True),
  StructField("study_time", LongType(), True),
  StructField("score", StringType(), True),
  StructField("is_challenge", LongType(), True),
  StructField("act_data", 
                ArrayType(
                  StructType([
                    StructField("act_code", StringType(), True),
                    StructField("act_name", StringType(), True),
                    StructField("act_order_seq", StringType(), True),
                    StructField("is_record_api", LongType(), True),
                    StructField("page_data", ArrayType(
                      StructType([
                        StructField("page_order_seq", StringType(), True),
                        StructField("rec_enable", LongType(), True),
                        StructField("model_answer", StringType(), True),
                        StructField("model_audio", StringType(), True),
                        StructField("std_audio", StringType(), True),
                        StructField("std_audio_time", DoubleType(), True),
                        StructField("std_audio_retry_cnt", LongType(), True),
                        StructField("std_answer", StringType(), True),
                        StructField("sr_score", StringType(), True),
                        StructField("sr_text", StructType([
                          StructField("EPM_sample_cnt", LongType(), True),
                          StructField("Intonation", ArrayType(
                            DoubleType()
                           ), True),
                          StructField("intonation_score", LongType(), True),
                          StructField("loudness_score", LongType(), True),
                          StructField("overall", LongType(), True),
                          StructField("pWord", ArrayType(
                            StructType([
                              StructField("bSmallLoudness", BooleanType(), True),
                              StructField("bSpoken", BooleanType(), True),
                              StructField("intonation", LongType(), True),
                              StructField("loudness", LongType(), True),
                              StructField("nStress", LongType(), True),
                              StructField("phoneme", StringType(), True),
                              StructField("phoneme_cnt", LongType(), True),
                              StructField("phoneme_position", ArrayType(
                                ArrayType(LongType(), True)
                              ), True),
                              StructField("position", ArrayType(
                                ArrayType(LongType(), True)
                              ), True),
                              StructField("pronunciation", LongType(), True),
                              StructField("timing", LongType(), True),
                            ])
                          ), True),
                          StructField("pronunciation_score", LongType(), True),
                          StructField("prosody_score", LongType(), True),
                          StructField("speed_score", DoubleType(), True),
                          StructField("subomission_score", DoubleType(), True),
                          StructField("timing_score", LongType(), True),
                          StructField("word_cnt", LongType(), True),
                        ]), True)
                      ]), True
                    ), True)
                  ]), True
                ), True  
  ),
  StructField("std_criteria_info", 
                ArrayType(
                  StructType([
                    StructField("try_cnt", LongType(), True),
                    StructField("is_active", LongType(), True),
                    StructField("std_criteria", ArrayType(
                      StructType([
                        StructField("scIdx", LongType(), True),
                        StructField("scsIdx", LongType(), True),
                        StructField("scdIdx", LongType(), True),
                        StructField("point", LongType(), True)
                      ]), True)
                    , True)
                  ])
                ), True)
])

In [5]:
shadowing_1046_DF = spark.read.format("json").schema(shadowing_1046_schema).load("./mnt/cdpocmount/april/study_result/544/790/93414/1046/*/*.txt")
shadowing_1046_DF.printSchema()

root
-- sem_id: long (nullable = true)
-- top_cors_id: long (nullable = true)
-- com_id: long (nullable = true)
-- g_seq: long (nullable = true)
-- app_ver: string (nullable = true)
-- serial: string (nullable = true)
-- mac_address: string (nullable = true)
-- manufacturer: string (nullable = true)
-- model: string (nullable = true)
-- submit_seq: string (nullable = true)
-- study_time: long (nullable = true)
-- score: string (nullable = true)
-- is_challenge: long (nullable = true)
-- act_data: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- act_code: string (nullable = true)
 | |-- act_name: string (nullable = true)
 | |-- act_order_seq: string (nullable = true)
 | |-- is_record_api: long (nullable = true)
 | |-- page_data: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- page_order_seq: string (nullable = true)
 | | | |-- rec_enable: long (nullable = true)
 | | | |-- model_answer: string (nullable = true)
 | | | |-- model_audio: string (nullable = true)
 | | | |-- std_audio: string (nullable = true)
 | | | |-- std_audio_time: double (nullable = true)
 | | | |-- std_audio_retry_cnt: long (nullable = true)
 | | | |-- std_answer: string (nullable = true)
 | | | |-- sr_score: string (nullable = true)
 | | | |-- sr_text: struct (nullable = true)
 | | | | |-- EPM_sample_cnt: long (nullable = true)
 | | | | |-- Intonation: array (nullable = true)
 | | | | | |-- element: double (containsNull = true)
 | | | | |-- intonation_score: long (nullable = true)
 | | | | |-- loudness_score: long (nullable = true)
 | | | | |-- overall: long (nullable = true)
 | | | | |-- pWord: array (nullable = true)
 | | | | | |-- element: struct (containsNull = true)
 | | | | | | |-- bSmallLoudness: boolean (nullable = true)
 | | | | | | |-- bSpoken: boolean (nullable = true)
 | | | | | | |-- intonation: long (nullable = true)
 | | | | | | |-- loudness: long (nullable = true)
 | | | | | | |-- nStress: long (nullable = true)
 | | | | | | |-- phoneme: string (nullable = true)
 | | | | | | |-- phoneme_cnt: long (nullable = true)
 | | | | | | |-- phoneme_position: array (nullable = true)
 | | | | | | | |-- element: array (containsNull = true)
 | | | | | | | | |-- element: long (containsNull = true)
 | | | | | | |-- position: array (nullable = true)
 | | | | | | | |-- element: array (containsNull = true)
 | | | | | | | | |-- element: long (containsNull = true)
 | | | | | | |-- pronunciation: long (nullable = true)
 | | | | | | |-- timing: long (nullable = true)
 | | | | |-- pronunciation_score: long (nullable = true)
 | | | | |-- prosody_score: long (nullable = true)
 | | | | |-- speed_score: double (nullable = true)
 | | | | |-- subomission_score: double (nullable = true)
 | | | | |-- timing_score: long (nullable = true)
 | | | | |-- word_cnt: long (nullable = true)
-- std_criteria_info: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- try_cnt: long (nullable = true)
 | |-- is_active: long (nullable = true)
 | |-- std_criteria: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- scIdx: long (nullable = true)
 | | | |-- scsIdx: long (nullable = true)
 | | | |-- scdIdx: long (nullable = true)
 | | | |-- point: long (nullable = true)

In [6]:
shadowing_1046_DF.createOrReplaceTempView("shadowing_temp_with_schema")

In [7]:
%sql
show tables;

database,tableName,isTemporary
default,chungdahm_619_prod,false
default,chungdahm_619_prod_test,false
default,chungdahm_619_prod_test2,false
default,shadowing_prod,false
default,tbl_cdi_branch,false
default,tbl_cdi_branch_semester,false
default,tbl_cdi_class,false
default,tbl_cdi_class_journal,false
default,tbl_cdi_class_student,false
default,tbl_cdi_evaluation_syllabus_detail,false


In [8]:
%sql
describe shadowing_temp_with_schema

col_name,data_type,comment
sem_id,bigint,null
top_cors_id,bigint,null
com_id,bigint,null
g_seq,bigint,null
app_ver,string,null
serial,string,null
mac_address,string,null
manufacturer,string,null
model,string,null
submit_seq,string,null


## 3. Delta Table 정의
- Wrangling 결과물을 저장하기 위한 별도 테이블
- Databricks의 delta lake를 활용

In [10]:
%sql
-- Product table 생성 (없을 경우 생성 )
create table IF NOT EXISTS shadowing_prod2(
std_id bigint 
, sem_id bigint
,top_cors_id	bigint	
,com_id bigint
,g_seq bigint
,app_ver string
,serial	string	
,mac_address string
,manufacturer	string
,model	string	
,submit_seq	string	
,study_time	bigint	
,score string
,is_challenge string

,act_data_act_code string 
,act_data_act_name string  
,act_data_act_order_seq string 
,act_data_is_record_api string 
,act_data_page_page_order_seq string
,act_data_page_rec_enable string
,act_data_page_model_answer string
,act_data_page_model_audio string
,act_data_page_std_audio string
,act_data_page_std_audio_time string
,act_data_page_audio_retry_cnt int
,act_data_page_std_answer string
,act_data_page_sr_score string

,act_data_page_sr_text_EPM_sample_cnt bigint
,act_data_page_sr_text_Intonation string 
,act_data_page_sr_text_intonation_score string
,act_data_page_sr_text_loudness_score string
,act_data_page_sr_text_overall string

,act_data_page_sr_text_pword_bSmallLoudness string
,act_data_page_sr_text_pword_bSpoken string
,act_data_page_sr_text_pword_intonation string
,act_data_page_sr_text_pword_loudness string
,act_data_page_sr_text_pword_nStress string
,act_data_page_sr_text_pword_phoneme string
,act_data_page_sr_text_pword_phoneme_cnt bigint
,act_data_page_sr_text_pword_phoneme_position string
,act_data_page_sr_text_pword_position string
,act_data_page_sr_text_pword_pronunciation string
,act_data_page_sr_text_pword_timing string

,act_data_page_sr_text_pronunciation_score bigint
,act_data_page_sr_text_prosody_score bigint
,act_data_page_sr_text_speed_score bigint
,act_data_page_sr_text_subomission_score bigint
,act_data_page_sr_text_timing_score bigint
,act_data_page_sr_text_word_cnt bigint

,std_criteria_info_data_try_cnt string 
,std_criteria_info_data_is_active string 
,std_criteria_info_data_scIdx string 
,std_criteria_info_data_scsIdx string 
,std_criteria_info_data_scdIdx string 
,std_criteria_info_data_point string 


)
USING DELTA
  LOCATION '/mnt/cdpocmount/delta/shadowing_prod2'

## 4. Delta Table에 wragling 결과물을 insert
- 폴더이름(std_id)를 별도의 컬럼으로 생성하여 저장
- JSON denormalizing
  - explode 함수를 활용

In [12]:
%sql
--###############################################################################
-- temp table 상으로 parsing 및 insert 

insert into shadowing_prod2
select 
split(input_file_name(), '/')[9] as std_id
, sem_id
, top_cors_id
, com_id
, g_seq
, app_ver
, serial
, mac_address
, manufacturer
, model
, submit_seq
, study_time
, score
, is_challenge

, act_data_data.act_code
, act_data_data.act_name
, act_data_data.act_order_seq
, act_data_data.is_record_api

, act_data_data_page_data.page_order_seq
, act_data_data_page_data.rec_enable
, act_data_data_page_data.model_answer
, act_data_data_page_data.model_audio
, act_data_data_page_data.std_audio
, act_data_data_page_data.std_audio_time
, act_data_data_page_data.std_audio_retry_cnt
, act_data_data_page_data.std_answer
, act_data_data_page_data.sr_score


--, act_data_data_page_data.sr_text
, act_data_data_page_data.sr_text.EPM_sample_cnt
, act_data_data_page_data.sr_text.Intonation
, act_data_data_page_data.sr_text.intonation_score
, act_data_data_page_data.sr_text.loudness_score
, act_data_data_page_data.sr_text.overall

, act_data_data_page_data_sr_text_pWord.bSmallLoudness
, act_data_data_page_data_sr_text_pWord.bSpoken
, act_data_data_page_data_sr_text_pWord.intonation
, act_data_data_page_data_sr_text_pWord.loudness
, act_data_data_page_data_sr_text_pWord.nStress
, act_data_data_page_data_sr_text_pWord.phoneme
, act_data_data_page_data_sr_text_pWord.phoneme_cnt
, act_data_data_page_data_sr_text_pWord.phoneme_position
, act_data_data_page_data_sr_text_pWord.position
, act_data_data_page_data_sr_text_pWord.pronunciation
, act_data_data_page_data_sr_text_pWord.timing

, act_data_data_page_data.sr_text.pronunciation_score
, act_data_data_page_data.sr_text.prosody_score
, act_data_data_page_data.sr_text.speed_score
, act_data_data_page_data.sr_text.subomission_score
, act_data_data_page_data.sr_text.timing_score
, act_data_data_page_data.sr_text.word_cnt

, std_criteria_info_data.try_cnt
, std_criteria_info_data.is_active
, std_criteria_info_data_std_criteria.scIdx
, std_criteria_info_data_std_criteria.scsIdx
, std_criteria_info_data_std_criteria.scdIdx
, std_criteria_info_data_std_criteria.point


from shadowing_temp_with_schema
lateral view explode(act_data) act_data_data as act_data_data
lateral view explode(act_data_data.page_data) act_data_data_page_data as act_data_data_page_data
lateral view explode(act_data_data_page_data.sr_text.pWord) act_data_data_page_data_sr_text_pWord as act_data_data_page_data_sr_text_pWord
lateral view explode(std_criteria_info) std_criteria_info_data as std_criteria_info_data
lateral view explode(std_criteria_info_data.std_criteria) std_criteria_info_data_std_criteria as std_criteria_info_data_std_criteria


/*
lateral view explode(evaluation_data) evaluation_data_data as evaluation_data_data
lateral view explode(study_data) study_data_data as study_data_data
lateral view explode(SR_data) SR_data_data as SR_data_data
lateral view explode(SR_data_data.data.analysisResult.wordResult) SR_data_wordResult as SR_data_wordResult
*/
--lateral view explode(SR_data_data.data) SR_data_data_data as SR_data_data_data

--LATERAL VIEW json_tuple(evaluation_data_data.jsonObject, 'data', 'element') v1 as data, element

## 5. Delta Table 결과물 확인

In [14]:
%sql
show tables

database,tableName,isTemporary
default,chungdahm_619_prod,false
default,chungdahm_619_prod_test,false
default,chungdahm_619_prod_test2,false
default,shadowing_prod,false
default,shadowing_prod2,false
default,tbl_cdi_branch,false
default,tbl_cdi_branch_semester,false
default,tbl_cdi_class,false
default,tbl_cdi_class_journal,false
default,tbl_cdi_class_student,false


In [15]:
%sql
select * from shadowing_prod2 limit 10;

std_id,sem_id,top_cors_id,com_id,g_seq,app_ver,serial,mac_address,manufacturer,model,submit_seq,study_time,score,is_challenge,act_data_act_code,act_data_act_name,act_data_act_order_seq,act_data_is_record_api,act_data_page_page_order_seq,act_data_page_rec_enable,act_data_page_model_answer,act_data_page_model_audio,act_data_page_std_audio,act_data_page_std_audio_time,act_data_page_audio_retry_cnt,act_data_page_std_answer,act_data_page_sr_score,act_data_page_sr_text_EPM_sample_cnt,act_data_page_sr_text_Intonation,act_data_page_sr_text_intonation_score,act_data_page_sr_text_loudness_score,act_data_page_sr_text_overall,act_data_page_sr_text_pword_bSmallLoudness,act_data_page_sr_text_pword_bSpoken,act_data_page_sr_text_pword_intonation,act_data_page_sr_text_pword_loudness,act_data_page_sr_text_pword_nStress,act_data_page_sr_text_pword_phoneme,act_data_page_sr_text_pword_phoneme_cnt,act_data_page_sr_text_pword_phoneme_position,act_data_page_sr_text_pword_position,act_data_page_sr_text_pword_pronunciation,act_data_page_sr_text_pword_timing,act_data_page_sr_text_pronunciation_score,act_data_page_sr_text_prosody_score,act_data_page_sr_text_speed_score,act_data_page_sr_text_subomission_score,act_data_page_sr_text_timing_score,act_data_page_sr_text_word_cnt,std_criteria_info_data_try_cnt,std_criteria_info_data_is_active,std_criteria_info_data_scIdx,std_criteria_info_data_scsIdx,std_criteria_info_data_scdIdx,std_criteria_info_data_point
1336855,544,790,1046,93414,,,,,,1,552,10,1,APIL_00521,Let's Record,2,1,576,1,Tom gets grapes.,https://ail.chungdahm.com/LCMSFiles/Packing_org/APRILIL/APIL_T01_R/790/Seed1/APRILIL_790_Seed1_93414_103/SD1_B1_L1_shdowing1.mp3,april\learning\544\790\1046\93414\1336855_APIL_00521_0_2.mp3,3.1346938775510202,2,Tom gets grapes.,,null,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09210173785686493, 0.4018566608428955, 0.7047519087791443, 0.8402842283248901, 0.923896312713623, 0.9678038358688354, 0.9492456316947937, 0.8365499973297119, 0.6817840933799744, 0.5940003395080566, 0.5110914707183838, 0.296501487493515, 0.1996029615402222, 0.4696890115737915, 0.7444301247596741, 0.7704344391822815, 0.7127283215522766, 0.6997116804122925, 0.7164929509162903, 0.6833637952804565, 0.5605899691581726, 0.3713209331035614, 0.1846343576908112, 0.07127001136541367, 0.07368990778923035, 0.2287755310535431, 0.5014272928237915, 0.6708943247795105, 0.5829442739486694, 0.32591512799263, 0.1186133325099945, 0.03953777998685837, 0.01317925937473774, 0.004393086768686771, 0.001464362372644246, 4.88120800582692E-4, 1.627069432288408E-4, 5.423565016826615E-5, 1.807855005608872E-5, 6.026183655194473E-6, 2.008727960856049E-6, 6.695759680042102E-7, 2.231920035455914E-7, 9.565371783537557E-8, 4.782685891768779E-8]",79,57,81,true,true,81,33,100,tam,3,"[[110, 850, 1070]]","[[110, 1250]]",85,84,77,81,76,100,88,3,1,1,8,15,67,3
1336855,544,790,1046,93414,,,,,,1,552,10,1,APIL_00521,Let's Record,2,1,576,1,Tom gets grapes.,https://ail.chungdahm.com/LCMSFiles/Packing_org/APRILIL/APIL_T01_R/790/Seed1/APRILIL_790_Seed1_93414_103/SD1_B1_L1_shdowing1.mp3,april\learning\544\790\1046\93414\1336855_APIL_00521_0_2.mp3,3.1346938775510202,2,Tom gets grapes.,,null,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09210173785686493, 0.4018566608428955, 0.7047519087791443, 0.8402842283248901, 0.923896312713623, 0.9678038358688354, 0.9492456316947937, 0.8365499973297119, 0.6817840933799744, 0.5940003395080566, 0.5110914707183838, 0.296501487493515, 0.1996029615402222, 0.4696890115737915, 0.7444301247596741, 0.7704344391822815, 0.7127283215522766, 0.6997116804122925, 0.7164929509162903, 0.6833637952804565, 0.5605899691581726, 0.3713209331035614, 0.1846343576908112, 0.07127001136541367, 0.07368990778923035, 0.2287755310535431, 0.5014272928237915, 0.6708943247795105, 0.5829442739486694, 0.32591512799263, 0.1186133325099945, 0.03953777998685837, 0.01317925937473774, 0.004393086768686771, 0.001464362372644246, 4.88120800582692E-4, 1.627069432288408E-4, 5.423565016826615E-5, 1.807855005608872